**POLARITY USING TEXTBLOB**

In [47]:
import pandas as pd
from textblob import TextBlob

In [48]:
df = pd.read_excel(r'./enSome.xlsx', sheet_name='Sheet1', index_col=0)

In [49]:
df

,Movie ID,Movie Title,User Rating,Review Title,Username,Review,Cleaned Review,Review Lang
2,tt0045997,Lewat Djam Malam (Restorasi 2011-2012),NaN,NaN,AdityaRassendrya,Terima Kasih Martin Scorsese yang telah memban...,thank you martin scorsese who has helped the p...,en
4,tt0045997,Lewat Djam Malam (Restorasi 2011-2012),NaN,NaN,SantoKlingon,Do'a yang terbaik untuk Mas Totot Alm ; Al-Fat...,the best prayer for the deceased mas totot al ...,en
5,tt0045997,Lewat Djam Malam (Restorasi 2011-2012),NaN,NaN,Vino,"Ya Allah, kursi perabotannya jd inget ibuku za...",oh god the furniture chair so remember my moth...,en
8,tt0045997,Lewat Djam Malam (Restorasi 2011-2012),NaN,NaN,KudaTerbang,salut akting2 bagus.,salute good acting,en
9,tt0045997,Lewat Djam Malam (Restorasi 2011-2012),NaN,NaN,HadiKurniawan,Lebih enak nonton film tahun 50an daripada sin...,better to watch movies in the fifties than soa...,en
...,...,...,...,...,...,...,...,...
20102,tt13393058,OFFICIAL TRAILER FILM BEN & JODY | SEDANG TAYA...,NaN,NaN,REBAHANMALAM,FILM CACAD HAHH,broken movie,en
20103,tt13393058,OFFICIAL TRAILER FILM BEN & JODY | SEDANG TAYA...,NaN,NaN,alex,Kkn,nepotism collusion corruption,en
20104,tt13393058,OFFICIAL TRAILER FILM BEN & JODY | SEDANG TAYA...,NaN,NaN,alex,Lkn,lkn,en
20105,tt13393058,OFFICIAL TRAILER FILM BEN & JODY | SEDANG TAYA...,NaN,NaN,BUNGFEI,Garing perasaan,feels boring,en


In [50]:
dictionary = []

for text in df['Cleaned Review']:
    sentiments = {}
    blob = TextBlob(str(text))
    sentiment = blob.sentiment.polarity
    if sentiment > 0.0:
      sentiments['Sentiments'] = 'positive'
    elif sentiment == 0.0:
      sentiments['Sentiments'] = 'neutral'
    else:
      sentiments['Sentiments'] = 'negative'

    dictionary.append(sentiments)
    print(sentiments)

{'Sentiments': 'neutral'}
{'Sentiments': 'positive'}
{'Sentiments': 'neutral'}
{'Sentiments': 'positive'}
{'Sentiments': 'positive'}
{'Sentiments': 'positive'}
{'Sentiments': 'positive'}
{'Sentiments': 'negative'}
{'Sentiments': 'positive'}
{'Sentiments': 'positive'}
{'Sentiments': 'positive'}
{'Sentiments': 'neutral'}
{'Sentiments': 'negative'}
{'Sentiments': 'neutral'}
{'Sentiments': 'positive'}
{'Sentiments': 'positive'}
{'Sentiments': 'positive'}
{'Sentiments': 'positive'}
{'Sentiments': 'positive'}
{'Sentiments': 'positive'}
{'Sentiments': 'neutral'}
{'Sentiments': 'neutral'}
{'Sentiments': 'neutral'}
{'Sentiments': 'neutral'}
{'Sentiments': 'neutral'}
{'Sentiments': 'negative'}
{'Sentiments': 'positive'}
{'Sentiments': 'neutral'}
{'Sentiments': 'positive'}
{'Sentiments': 'neutral'}
{'Sentiments': 'positive'}
{'Sentiments': 'positive'}
{'Sentiments': 'positive'}
{'Sentiments': 'neutral'}
{'Sentiments': 'neutral'}
{'Sentiments': 'positive'}
{'Sentiments': 'positive'}
{'Sentiments':

In [51]:
polar_textblob = pd.DataFrame(dictionary)
polar_textblob

,Sentiments
0,neutral
1,positive
2,neutral
3,positive
4,positive
...,...
3690,negative
3691,neutral
3692,neutral
3693,negative


In [53]:
polar_textblob.to_excel('polar_textblob.xlsx', index=False)

**POLARITY USING ROBERTA**

In [1]:
import pandas as pd
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
from tqdm.notebook import tqdm

In [2]:
df = pd.read_excel('enSome.xlsx')
df.rename(columns={'Unnamed: 0': 'Id'}, inplace=True)
df['Cleaned Review'] = df['Cleaned Review'].astype(str)
df.head()

,Id,Movie ID,Movie Title,User Rating,Review Title,Username,Review,Cleaned Review,Review Lang
0,2,tt0045997,Lewat Djam Malam (Restorasi 2011-2012),NaN,NaN,AdityaRassendrya,Terima Kasih Martin Scorsese yang telah memban...,thank you martin scorsese who has helped the p...,en
1,4,tt0045997,Lewat Djam Malam (Restorasi 2011-2012),NaN,NaN,SantoKlingon,Do'a yang terbaik untuk Mas Totot Alm ; Al-Fat...,the best prayer for the deceased mas totot al ...,en
2,5,tt0045997,Lewat Djam Malam (Restorasi 2011-2012),NaN,NaN,Vino,"Ya Allah, kursi perabotannya jd inget ibuku za...",oh god the furniture chair so remember my moth...,en
3,8,tt0045997,Lewat Djam Malam (Restorasi 2011-2012),NaN,NaN,KudaTerbang,salut akting2 bagus.,salute good acting,en
4,9,tt0045997,Lewat Djam Malam (Restorasi 2011-2012),NaN,NaN,HadiKurniawan,Lebih enak nonton film tahun 50an daripada sin...,better to watch movies in the fifties than soa...,en


In [3]:
MODEL = f'cardiffnlp/twitter-roberta-base-sentiment'
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
example = df['Cleaned Review'].loc[30]
example

'this is a very good movie I have watched every cynthia rothrock movies'

In [7]:
encoded_text = tokenizer(example, return_tensors='pt')
output = model(**encoded_text)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores_dict = {
    'roberta_neg' : scores[0],
    'roberta_neu' : scores[1],
    'roberta_pos' : scores[2]
}

print(scores_dict)

{'roberta_neg': 0.0015271947, 'roberta_neu': 0.0114044, 'roberta_pos': 0.9870684}


In [9]:
def evaluate_sentiment(row):
    roberta_neg = row['roberta_neg']
    roberta_pos = row['roberta_pos']
    roberta_neu = row['roberta_neu']

    if roberta_neg > roberta_pos and roberta_neg > roberta_neu:
        return 'negative'
    elif roberta_pos > roberta_neg and roberta_pos > roberta_neu:
        return 'positive'
    else:
        return 'neutral'

result = evaluate_sentiment(scores_dict)
print("Sentiment value:", result)

Sentiment value: positive


In [10]:
def polarity_scores_roberta(example):
  encoded_text = tokenizer(example, return_tensors='pt')
  output = model(**encoded_text)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  scores_dict = {
      'roberta_neg' : scores[0],
      'roberta_neu' : scores[1],
      'roberta_pos' : scores[2]
  }
  return scores_dict

In [11]:
res = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
  try:
    text = row['Cleaned Review']
    myid = row['Id']
    roberta_result = polarity_scores_roberta(text)
    res[myid] = roberta_result
  except RuntimeError:
    print(f'Broke for id {myid}')

  0%|          | 0/3695 [00:00<?, ?it/s]

In [14]:
results_df = pd.DataFrame(res).T
results_df

,roberta_neg,roberta_neu,roberta_pos
2,0.003039,0.071220,0.925740
4,0.020959,0.481161,0.497880
5,0.206634,0.706054,0.087312
8,0.010009,0.099157,0.890834
9,0.311765,0.579117,0.109118
...,...,...,...
20102,0.267075,0.591220,0.141706
20103,0.781175,0.207743,0.011082
20104,0.266979,0.501644,0.231376
20105,0.922899,0.066553,0.010548


In [34]:
def evaluate_overall_sentiment(dictionary):
    result_dict = {}
    for key, row in dictionary.items():
        roberta_neg = row.get('roberta_neg')
        roberta_pos = row.get('roberta_pos')
        roberta_neu = row.get('roberta_neu')

        if roberta_neg > roberta_pos and roberta_neg > roberta_neu:
            result_dict[key] = 'negative'
        elif roberta_pos > roberta_neg and roberta_pos > roberta_neu:
            result_dict[key] = 'positive'
        else:
            result_dict[key] = 'neutral'
    return result_dict

overall_result = evaluate_overall_sentiment(res)
print(overall_result)

{2: 'positive', 4: 'positive', 5: 'neutral', 8: 'positive', 9: 'neutral', 10: 'positive', 12: 'neutral', 13: 'neutral', 14: 'neutral', 15: 'neutral', 16: 'neutral', 17: 'neutral', 18: 'neutral', 19: 'neutral', 20: 'positive', 21: 'neutral', 22: 'neutral', 23: 'neutral', 24: 'neutral', 25: 'neutral', 26: 'neutral', 27: 'neutral', 28: 'neutral', 29: 'neutral', 30: 'neutral', 31: 'negative', 32: 'neutral', 33: 'neutral', 34: 'positive', 35: 'neutral', 122: 'positive', 128: 'positive', 136: 'positive', 139: 'neutral', 144: 'neutral', 167: 'negative', 168: 'neutral', 172: 'neutral', 173: 'neutral', 174: 'positive', 175: 'neutral', 176: 'positive', 177: 'positive', 178: 'negative', 179: 'neutral', 180: 'positive', 181: 'neutral', 182: 'positive', 183: 'neutral', 184: 'negative', 185: 'positive', 186: 'neutral', 187: 'neutral', 188: 'neutral', 189: 'negative', 190: 'positive', 192: 'positive', 193: 'negative', 194: 'negative', 195: 'negative', 196: 'neutral', 197: 'neutral', 198: 'negative', 

In [45]:
overall_results_df = pd.DataFrame(overall_result.items(), columns=['Id', 'sentiment'])
overall_results_df.set_index('Id', inplace=True)
overall_results_df.index.name = None
overall_results_df

,sentiment
2,positive
4,positive
5,neutral
8,positive
9,neutral
...,...
20102,neutral
20103,negative
20104,neutral
20105,negative


In [46]:
overall_results_df.to_excel('polar_roberta.xlsx', index=False)